In [1]:
from osgeo import gdal

In [2]:
step1 = gdal.Open('data/20180116/0_B02_20180116.tif', gdal.GA_ReadOnly)

In [3]:
GT_input = step1.GetGeoTransform()

In [4]:
print(GT_input)

(693559.1091246363, 10.0, 0.0, 133265.48983932866, 0.0, -10.0)


In [6]:
from affine import Affine

In [7]:
transf = (693559.1091246363, 10.0, 0.0, 133265.48983932866, 0.0, -10.0)
afn = Affine.from_gdal(*transf)

In [8]:
step2 = step1.GetRasterBand(1)

In [9]:
img_as_array = step2.ReadAsArray()

In [10]:
x =img_as_array

In [11]:
size1,size2=img_as_array.shape

In [12]:
import numpy as np
output=np.zeros(shape=(size1,size2))

In [13]:
output.shape

(4251, 3370)

In [14]:
x_flatten = x.ravel()

In [15]:
print("New resulting array: ", x_flatten)

New resulting array:  [0 0 0 ... 0 0 0]


In [16]:
x_flatten.shape

(14325870,)

In [17]:
Field_id = np.arange(len(x_flatten))

In [18]:
result = np.column_stack((x_flatten,Field_id))

In [19]:
print ("resultant array", str(result))

resultant array [[       0        0]
 [       0        1]
 [       0        2]
 ...
 [       0 14325867]
 [       0 14325868]
 [       0 14325869]]


In [20]:
result_deleted = np.delete(result, 0, 1)

In [21]:
len(result_deleted) 

14325870

In [22]:
len(np.unique(result_deleted))

14325870

In [23]:
print ("resultant array", str(result_deleted))

resultant array [[       0]
 [       1]
 [       2]
 ...
 [14325867]
 [14325868]
 [14325869]]


In [24]:
array_output = np.reshape(result_deleted, (4251, 3370))

In [25]:
array_output.shape

(4251, 3370)

In [26]:
import rasterio

In [27]:
dst_crs='EPSG:32636'

In [28]:
output = np.int32(array_output)

In [29]:
with rasterio.open(
    'field_id.tif',
    'w',
    driver='GTiff',
    height=output.shape[0],
    width=output.shape[1],
    count=1,
    dtype=np.int32,
    crs=dst_crs,
    transform=afn,
) as dest_file:
    dest_file.write(output, 1)
dest_file.close()